# 1. Model Training

In [38]:
import os

def check_balance(data_dir='./data'):
    counts = {}
    folders = []  
    
    for class_dir in os.listdir(data_dir):
        class_path = os.path.join(data_dir, class_dir)
        if os.path.isdir(class_path):
            folders.append(class_dir)
            counts[class_dir] = len(os.listdir(class_path))
    
    print("Counts:", counts)
    print("Ratios:", {k: f"{(counts[k]/sum(counts.values())*100):.1f}%" for k in folders})

check_balance()

Counts: {'Finished': 438, 'Opened': 287, 'Sealed': 314}
Ratios: {'Finished': '42.2%', 'Opened': '27.6%', 'Sealed': '30.2%'}


In [36]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Config - easy to change
CONFIG = {
    'IMG_SIZE': (75, 75),
    'BATCH_SIZE': 32,  # Smaller, dataset-agnostic
    'EPOCHS': 20,
    'NUM_CLASSES': 3,
    'DATA_DIR': './data'
}

class_names = ['Finished', 'Opened', 'Sealed']

def check_dataset_balance(data_dir):
    """Verify class distribution before training."""
    counts = {}
    for class_dir in sorted(os.listdir(data_dir)):  # Consistent order
        class_path = os.path.join(data_dir, class_dir)
        if os.path.isdir(class_path):
            counts[class_dir] = len([f for f in os.listdir(class_path) 
                                   if f.lower().endswith(('.jpg', '.png', '.jpeg'))])
    
    total = sum(counts.values())
    print("Dataset Balance:")
    print(counts)
    print("Ratios:", {k: f"{v/total*100:.1f}%" for k,v in counts.items()})
    return counts

def get_image_generators(data_dir, img_size, batch_size):
    """Simplified 80/10/10 split - more reliable than validation_split trick."""
    seed = 42
    
    # Train: augmentation
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split=0.2  # 80/20 split
    )
    
    train_gen = train_datagen.flow_from_directory(
        data_dir, target_size=img_size, batch_size=batch_size,
        class_mode='categorical', subset='training', seed=seed
    )
    
    # Val/Test: no augmentation
    val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.125)  # 10% val, 10% test
    
    val_gen = val_datagen.flow_from_directory(
        data_dir, target_size=img_size, batch_size=batch_size,
        class_mode='categorical', subset='training', seed=seed, shuffle=False
    )
    
    test_gen = val_datagen.flow_from_directory(
        data_dir, target_size=img_size, batch_size=batch_size,
        class_mode='categorical', subset='validation', seed=seed, shuffle=False
    )
    
    return train_gen, val_gen, test_gen

def build_improved_model(img_shape, num_classes):
    """Better architecture: GlobalAvgPool2D + more layers."""
    base_model = InceptionV3(input_shape=(*img_shape, 3), include_top=False, weights='imagenet')
    
    # Freeze base
    base_model.trainable = False
    
    # Better head (no Flatten = less params)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.3)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

# MAIN TRAINING PIPELINE
print("🔍 Checking dataset...")
check_dataset_balance(CONFIG['DATA_DIR'])

print("\n📊 Loading data...")
train_gen, val_gen, test_gen = get_image_generators(
    CONFIG['DATA_DIR'], CONFIG['IMG_SIZE'], CONFIG['BATCH_SIZE']
)

print("\n🏗️ Building model...")
model = build_improved_model(CONFIG['IMG_SIZE'], CONFIG['NUM_CLASSES'])
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
# model.summary()

# Compute class weights for imbalance (52%/35%/13% → weights adjust)
print("\n⚖️ Computing class weights...")
class_weights = compute_class_weight(
    'balanced', classes=np.unique(train_gen.classes), y=train_gen.classes
)
class_weight_dict = dict(enumerate(class_weights))
print("Class weights:", class_weight_dict)

# Callbacks
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True, monitor='val_accuracy'),
    ReduceLROnPlateau(factor=0.2, patience=3, min_lr=1e-7)
]

print("\n🚀 Training...")
history = model.fit(
    train_gen,
    epochs=CONFIG['EPOCHS'],
    validation_data=val_gen,
    class_weight=class_weight_dict,  # Handles your imbalance!
    callbacks=callbacks,
    verbose=1
)

print("\n✅ Test accuracy:", model.evaluate(test_gen)[1])

# Save with timestamp
import datetime
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
model.save(f'./models/cup_noodle_classifier_{timestamp}.keras')
print(f"💾 Saved: cup_noodle_classifier_{timestamp}.keras")

🔍 Checking dataset...
Dataset Balance:
{'Finished': 80, 'Opened': 244, 'Sealed': 314}
Ratios: {'Finished': '12.5%', 'Opened': '38.2%', 'Sealed': '49.2%'}

📊 Loading data...
Found 512 images belonging to 3 classes.
Found 559 images belonging to 3 classes.
Found 79 images belonging to 3 classes.

🏗️ Building model...


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 75, 75, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_282 (Conv2D) │ (None, 37, 37,    │        864 │ input_layer_3[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 37, 37,    │         96 │ conv2d_282[0][0]  │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_282      │ (None, 37, 37,    │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_283 (Conv2D) │ (None, 35, 35,    │      9,216 │ activation_282[0… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 35, 35,    │         96 │ conv2d_283[0][0]  │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_283      │ (None, 35, 35,    │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_284 (Conv2D) │ (None, 35, 35,    │     18,432 │ activation_283[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 35, 35,    │        192 │ conv2d_284[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_284      │ (None, 35, 35,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_12    │ (None, 17, 17,    │          0 │ activation_284[0… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_285 (Conv2D) │ (None, 17, 17,    │      5,120 │ max_pooling2d_12… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 17, 17,    │        240 │ conv2d_285[0][0]  │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_285      │ (None, 17, 17,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_286 (Conv2D) │ (None, 15, 15,    │    138,240 │ activation_285[0… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 15, 15,    │        576 │ conv2d_286[0][0]  │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_286      │ (None, 15, 15,    │          0 │ batch_normalizat

 Total params: 22,328,099 (85.17 MB)

 Trainable params: 525,315 (2.00 MB)

 Non-trainable params: 21,802,784 (83.17 MB)


⚖️ Computing class weights...
Class weights: {0: np.float64(2.6666666666666665), 1: np.float64(0.8707482993197279), 2: np.float64(0.6772486772486772)}

🚀 Training...
Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 59s 3s/step - accuracy: 0.6289 - loss: 1.1121 - val_accuracy: 0.8640 - val_loss: 0.3591 - learning_rate: 0.0010
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 40s 3s/step - accuracy: 0.7910 - loss: 0.5820 - val_accuracy: 0.8801 - val_loss: 0.3057 - learning_rate: 0.0010
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 42s 3s/step - accuracy: 0.7910 - loss: 0.6247 - val_accuracy: 0.8855 - val_loss: 0.2980 - learning_rate: 0.0010
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 43s 3s/step - accuracy: 0.7910 - loss: 0.5792 - val_accuracy: 0.8640 - val_loss: 0.3190 - learning_rate: 0.0010
Epoch 5/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 42s 3s/step - accuracy: 0.8086 - loss: 0.4721 - val_accuracy: 0.8533 - val_loss: 0.3226 - learning_rate: 0.0010
Epoch 6/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 41s 3s/step - accuracy: 0.8125 - loss: 0.4880

### ⭐ Save the Model as a .keras File (so it can be reused later)

In [33]:
import os
import sys

if not model: 
    print("There's no model yet!")
    sys.exit()  

# get id for new model
folder = "./models/"
id = len([f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))])
default_filename = f"model{id}"

new_model_filename = input(f"Enter filename (Default: {default_filename}): ")
if new_model_filename.strip() == "":
    new_model_filename = default_filename
tf.keras.models.save_model(model, f'./models/{new_model_filename}.keras')

# 2. Trying the Model

### 🔃 Load a .keras model
Loading a model is necessary if the above code to train a new model was not executed.
(Skip this if model was recently trained above)

In [39]:
import os
import tensorflow as tf

folder = "./models/"
files = os.listdir(folder)

# sort by last added time
if not files: raise FileNotFoundError("No models found!")
files_sorted = sorted(files, key=lambda f: os.path.getmtime(os.path.join(folder, f)))
last_added_filename = files_sorted[-1]

while True:
    input_filename = input(
        f"Enter model to load (Latest: {last_added_filename.replace('.keras', '')}): "
    ).strip()

    if input_filename == "":
        model_filename = last_added_filename
        break

    model_filename = f"{input_filename.replace('.keras', '')}.keras"
    full_path = os.path.join(folder, model_filename)

    if os.path.exists(full_path):
        break
    else:
        print("❌ Model not found. Please try again.")

model = tf.keras.models.load_model(f'./models/{model_filename}')

print("✅ Model Loaded:", model_filename)

✅ Model Loaded: cup_noodle_classifier_20260128_021448.keras


### Part 2A: Inference using Tensorflow Camera
(Please view the cell under this one to terminate the camera)

In [41]:
import threading
import cv2
import numpy as np
from PIL import Image, ImageOps
import tensorflow as tf

def import_and_predict(image_data):
    image = ImageOps.fit(image_data, (75, 75), Image.Resampling.LANCZOS)
    image = image.convert('RGB')
    image = np.asarray(image)
    image = (image.astype(np.float32) / 255.0)
    img_reshape = image[np.newaxis,...]
    prediction = model.predict(img_reshape, verbose=0)
    return prediction

stop_flag = False

def camera_loop():
    global stop_flag
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        cap.open()
        
    print("Camera Started!")

    while not stop_flag:
        ret, original = cap.read()
        if not ret:
            break

        image = Image.fromarray(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
        prediction = import_and_predict(image)

        map = {0: "Finished",
               1: "Opened",
               2: "Sealed"}

        idx = np.argmax(prediction)
        confidence = prediction[0][idx] 

        predict = f"It is a {map[idx]}! ({confidence:.2f})"

        cv2.putText(original, predict, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        cv2.imshow("Cup Noodles Detector", original)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            stop_flag = True

    cap.release()
    cv2.destroyAllWindows()

# start the camera in a separate thread
camera_thread = threading.Thread(target=camera_loop)
camera_thread.start()


Camera Started!


✋ Run to terminate the camera!
   👇

In [8]:
stop_flag = True
camera_thread.join()
print("Stopped!")

Stopped!


### Part 2B: Inference using Folder of Images
For this part, we will classify your images based on uploaded images

In [ ]:
import tkinter as tk
from tkinter import filedialog
import cv2
import numpy as np
from matplotlib import pyplot as plt
import math

COLUMNS = 5

def select_images_with_tkinter():
    """
    Opens tkinter file dialog to select multiple image files.
    Returns list of full file paths.
    """
    root = tk.Tk()
    root.withdraw()  # Hide the main window
    
    # Open file dialog for multiple images
    file_paths = filedialog.askopenfilenames(
        title="Select images to predict",
        filetypes=[
            ("Image files", "*.jpg *.jpeg *.png *.bmp *.tiff *.gif"),
            ("All files", "*.*")
        ]
    )
    
    root.destroy()
    return list(file_paths)

def visualize_predictions_from_files(image_paths: list, class_names: list[str]) -> None:
    """
    Visualize predictions from a list of image file paths (from tkinter).
    """
    if not image_paths:
        print("No images selected.")
        return
    
    rows = math.ceil(len(image_paths) / COLUMNS)
    plt.figure(figsize=(COLUMNS * 4, rows * 4))
    
    for i, img_path in enumerate(image_paths):
        # Load image
        img = cv2.imread(img_path)
        if img is None:
            print(f"⚠️ Could not load {img_path}")
            continue
            
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Preprocess
        img_resized = cv2.resize(img, (75, 75))
        img_norm = img_resized / 255.0
        img_input = np.expand_dims(img_norm, axis=0)
        
        # Predict
        preds = model.predict(img_input, verbose=0)
        class_id = np.argmax(preds[0])
        confidence = preds[0][class_id]
        filename = os.path.basename(img_path)  # Just filename for title
        label = f"{class_names[class_id]} ({confidence:.2f})"
        
        # Plot
        plt.subplot(rows, COLUMNS, i + 1)
        plt.imshow(img)
        plt.title(f"{filename[:20]}...\n{label}", fontsize=10)
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

# Usage - replaces the old line
class_names = ['Finished', 'Opened', 'Sealed']  # Update this for cup noodles!
image_files = select_images_with_tkinter()
visualize_predictions_from_files(image_files, class_names)
